Целью данного проекта является разработка модели предсказания стоимости автомобиля на вторичном рынке.

В качестве обучающих данных имеется информация о продажах (~440000) автомобилей с аукционов.

# Загрузка данных

Импортируем библиотеки:

In [172]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from fuzzywuzzy import fuzz
from statistics import mode
import jellyfish
from sklearn.impute import SimpleImputer
import matplotlib.ticker as ticker
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import ShuffleSplit
from vininfo import Vin

pd.set_option('display.float_format', '{:,.0f}'.format)
pd.options.mode.chained_assignment = None

Загрузим данные:

In [173]:
train = pd.read_csv('/Users/a.babaev/Desktop/kaggle/train.csv')
test = pd.read_csv('/Users/a.babaev/Desktop/kaggle/test.csv')

In [174]:
display(train.head())

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,sellingprice,saledate
0,2011,Ford,Edge,SEL,suv,automatic,2fmdk3jc4bba41556,md,4,"111,041",black,black,santander consumer,12500,Tue Jun 02 2015 02:30:00 GMT-0700 (PDT)
1,2014,Ford,Fusion,SE,Sedan,automatic,3fa6p0h75er208976,mo,4,"31,034",black,black,ars/avis budget group,14500,Wed Feb 25 2015 02:00:00 GMT-0800 (PST)
2,2012,Nissan,Sentra,2.0 SL,sedan,automatic,3n1ab6ap4cl698412,nj,2,"35,619",black,black,nissan-infiniti lt,9100,Wed Jun 10 2015 02:30:00 GMT-0700 (PDT)
3,2003,HUMMER,H2,Base,suv,automatic,5grgn23u93h101360,tx,3,"131,301",gold,beige,wichita falls ford lin inc,13300,Wed Jun 17 2015 03:00:00 GMT-0700 (PDT)
4,2007,Ford,Fusion,SEL,Sedan,automatic,3fahp08z17r268380,md,2,"127,709",black,black,purple heart,1300,Tue Feb 03 2015 04:00:00 GMT-0800 (PST)


Согласно документации к данным:

- `year` - год производства
- `make` - производитель
- `model` - модель
- `trim` - модификация
- `body` - тип кузова
- `transmission` - тип КПП
- `vin` - идентификатор
- `state` - штат регистрации
- `condition` - состояние по шкале (1-5)
- `odometer` - пробег в милях
- `color` - цвет кузова
- `interior` - цвет интерьера
- `seller` - продавец
- `sellingprice` - стоимость продажи
- `saledate` - дата продажи

Изучим общую информацию о таблицах:

In [175]:
dfs = [train, test]
dfs_name = ['train', 'test']

for i, j in zip(dfs, dfs_name):
    print(j, ':')
    i.info()
    print('\n')

train :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440236 entries, 0 to 440235
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   year          440236 non-null  int64  
 1   make          432193 non-null  object 
 2   model         432113 non-null  object 
 3   trim          431899 non-null  object 
 4   body          429843 non-null  object 
 5   transmission  388775 non-null  object 
 6   vin           440236 non-null  object 
 7   state         440236 non-null  object 
 8   condition     430831 non-null  float64
 9   odometer      440167 non-null  float64
 10  color         439650 non-null  object 
 11  interior      439650 non-null  object 
 12  seller        440236 non-null  object 
 13  sellingprice  440236 non-null  int64  
 14  saledate      440236 non-null  object 
dtypes: float64(2), int64(2), object(11)
memory usage: 50.4+ MB


test :
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110060 ent

In [176]:
dfs = [train, test]
dfs_name = ['train', 'test']

for i, j in zip(dfs, dfs_name):
    print('Пропуски в таблице ', j, ':', sep='')
    display(i.isna().sum())
    print('\n')

Пропуски в таблице train:


year                0
make             8043
model            8123
trim             8337
body            10393
transmission    51461
vin                 0
state               0
condition        9405
odometer           69
color             586
interior          586
seller              0
sellingprice        0
saledate            0
dtype: int64



Пропуски в таблице test:


year                0
make             2061
model            2079
trim             2114
body             2594
transmission    13012
vin                 0
state               0
condition        2379
odometer           19
color             158
interior          158
seller              0
saledate            0
dtype: int64

В датафреймах присутствуют пропуски данных.

Проверим наличие дубликатов в датафреймах:

In [177]:
for i, j in zip(dfs, dfs_name):
    print('Количество дубликатов в таблице ', j,': ', i.duplicated().sum(), sep='')

Количество дубликатов в таблице train: 0
Количество дубликатов в таблице test: 0


Явные дубликаты отсутствуют.

Изучим распределение количественных переменных:

In [178]:
for i, j in zip(dfs, dfs_name):
    print(j, ':', sep='')
    display(i.describe().T)

train:


,count,mean,std,min,25%,50%,75%,max
year,"440,236","2,010",4,"1,982","2,007","2,012","2,013","2,015"
condition,"430,831",3,1,1,3,4,4,5
odometer,"440,167","68,344","53,542",1,"28,258","52,098","99,272","999,999"
sellingprice,"440,236","13,592","9,751",1,"6,900","12,100","18,200","230,000"


test:


,count,mean,std,min,25%,50%,75%,max
year,"110,060","2,010",4,"1,982","2,007","2,012","2,013","2,015"
condition,"107,681",3,1,1,3,4,4,5
odometer,"110,041","68,076","53,524",1,"28,314","51,922","98,854","999,999"


## Предобработка данных

Объединим данные в единый датафрейм для предобработки данных:

In [179]:
df = pd.concat([train, test], keys=['train', 'test']).reset_index().drop(columns=['level_1']).rename(columns={'level_0':'data_type'})

In [180]:
# приведем столбец с датой в формат даты:
df['saledate'] = pd.to_datetime(df['saledate'], utc=True).dt.tz_localize(None)

# добавим столбцы с годом и месяцем продажи:
df['sale_year'] = df['saledate'].dt.year
df['sale_month'] = df['saledate'].dt.month

# приведем категориальный переменные к нижнему регистру :
cat_features = df.select_dtypes(include=['object']).columns.to_list()
cat_features.remove('data_type')
cat_features.remove('vin')
cat_features.remove('seller')
df[cat_features] = df[cat_features].apply(lambda x: x.str.lower())
df[cat_features] = df[cat_features].apply(lambda x: x.str.replace(" ", ""))

In [181]:
# унифицируем значения в столбце body:
df.loc[df['body'].str.contains('convert', na=False), 'body'] = 'convertible'
df.loc[df['body'].str.contains('sedan', na=False), 'body'] = 'sedan'
df.loc[df['body'].str.contains('cab', na=False), 'body'] = 'cab'
df.loc[df['body'].str.contains('van', na=False), 'body'] = 'van'
df.loc[df['body'].str.contains('wagon', na=False), 'body'] = 'wagon'
df.loc[df['body'].str.contains('coupe', na=False), 'body'] = 'coupe'
df.loc[df['body'].str.contains('koup', na=False), 'body'] = 'coupe'

# унифицируем значения в столбце make:
df.loc[df['make'].str.contains('fordtruck', na=False), 'make'] = 'ford' 
df.loc[df['make'].str.contains('chevtruck', na=False), 'make'] = 'chevrolet'
df.loc[df['make'].str.contains('gmctruck', na=False), 'make'] = 'gmc'
df.loc[df['make'].str.contains('vw', na=False), 'make'] = 'volkswagen'
df.loc[df['make'].str.contains('mercedes-benz', na=False), 'make'] = 'mercedes'

# заменим '-' на пропуски:
df['color'] = df['color'].replace('—', np.nan)
df['interior'] = df['interior'].replace('—', np.nan)

In [182]:
# используем wmi-код из vin номера для заполнения пропусков в столбце make:
print('Количество пропусков в столбце make:', df['make'].isna().sum())

df['wmi_code'] = df['vin'].str[:3]
wmi_dict = df.groupby(['wmi_code', 'make'], as_index=False).agg({'vin':'count'})
df.loc[df['make'].isna(), 'make'] = df[df['make'].isna()]['wmi_code'].map(dict(zip(wmi_dict['wmi_code'], wmi_dict['make'])))
df = df.drop('wmi_code', axis=1)

print('Количество пропусков в столбце make:', df['make'].isna().sum())

Количество пропусков в столбце make: 10104
Количество пропусков в столбце make: 22


In [183]:
# для заполнения оставшихся пропусков используем библиотеку vininfo:
vin = []
manufacture = []

for i in df[df['make'].isna()]['vin'].unique():
    vin.append(i)
    manufacture.append(Vin(i).manufacturer)
    
manufacture_dict = pd.DataFrame(list(zip(vin, manufacture)),columns=['vin', 'make'])
manufacture_dict['make'] = manufacture_dict['make'].str.lower().str.replace(" ", "")
df.loc[df['make'].isna(), 'make'] = df[df['make'].isna()]['vin'].map(dict(zip(manufacture_dict['vin'], manufacture_dict['make'])))

print('Количество пропусков в столбце make:', df['make'].isna().sum())

Количество пропусков в столбце make: 0


In [184]:
# используем wmi-код и код модели из vin номера для заполнения пропусков в столбце model:
print('Количество пропусков в столбце model:', df['model'].isna().sum())

df['model_code'] = df['vin'].str[:5]
model_dict = df.groupby(['model_code', 'model'], as_index=False).agg({'vin':'count'})
df.loc[df['model'].isna(), 'model'] = df[df['model'].isna()]['model_code'].map(dict(zip(model_dict['model_code'], model_dict['model'])))
df = df.drop('model_code', axis=1)

print('Количество пропусков в столбце model:', df['model'].isna().sum())

Количество пропусков в столбце model: 10202
Количество пропусков в столбце model: 176


In [185]:
# используем wmi-код и код модели из vin номера для заполнения пропусков в столбце trim:
print('Количество пропусков в столбце trim:', df['trim'].isna().sum())

df['trim_code'] = df['vin'].str[:6]
trim_dict = df.groupby(['trim_code', 'trim'], as_index=False).agg({'vin':'count'})
df.loc[df['trim'].isna(), 'trim'] = df[df['trim'].isna()]['trim_code'].map(dict(zip(trim_dict['trim_code'], trim_dict['trim'])))
df = df.drop('trim_code', axis=1)

print('Количество пропусков в столбце trim:', df['trim'].isna().sum())

Количество пропусков в столбце trim: 10451
Количество пропусков в столбце trim: 348


In [186]:
# заполним пропуски в столбце transmission наиболее популярным типом трансмиссии для модели:
print('Количество пропусков в столбце transmission:', df['transmission'].isna().sum())

transmission_dict = df.groupby(['model', 'transmission'], as_index=False).agg({'vin':'count'}).sort_values(by='vin', ascending=False)
transmission_dict = transmission_dict.drop_duplicates(subset='model', keep='first')
df.loc[df['transmission'].isna(), 'transmission'] = df[df['transmission'].isna()]['model'].map(dict(zip(transmission_dict['model'], transmission_dict['transmission'])))

print('Количество пропусков в столбце transmission:', df['transmission'].isna().sum())

Количество пропусков в столбце transmission: 64473
Количество пропусков в столбце transmission: 105


In [187]:
# заполним оставшиеся пропуски наиболее популярным типом трансмиссии для года выпуска:
print('Количество пропусков в столбце transmission:', df['transmission'].isna().sum())

for year in df[df['transmission'].isna()]['year'].unique(): 
    mode_transmission = df.loc[df['year'] == year, 'transmission'].mode()[0]
    df.loc[(df['transmission'].isna()) & (df['year'] == year), 'transmission'] = mode_transmission

print('Количество пропусков в столбце transmission:', df['transmission'].isna().sum())

Количество пропусков в столбце transmission: 105
Количество пропусков в столбце transmission: 0


In [188]:
# заполним пропуски в столбце odometer средним пробегом по автомобилям аналогичного года выпуска:
print('Количество пропусков в столбце odometer:', df['odometer'].isna().sum())

for year in df[df['odometer'].isna()]['year'].unique(): 
    avg_distance = df.loc[df['year'] == year, 'odometer'].mean()
    df.loc[(df['odometer'].isna()) & (df['year'] == year), 'odometer'] = avg_distance

print('Количество пропусков в столбце odometer:', df['odometer'].isna().sum())

Количество пропусков в столбце odometer: 88
Количество пропусков в столбце odometer: 0


In [189]:
# заполним пропуски в столбце condition средним значением по автомобилям аналогичным пробегом:
print('Количество пропусков в столбце condition:', df['condition'].isna().sum())

for distance in round(df['odometer'], -3).unique():

    avg_condition = df.loc[round(df['odometer'], -3) == round(distance, -3), 'condition'].mean()
    df.loc[(df['condition'].isna()) & (round(df['odometer'], -3) == round(distance, -3)), 'condition'] = avg_condition 
    
    avg_condition = df.loc[round(df['odometer'], -4) == round(distance, -4), 'condition'].mean()
    df.loc[(df['condition'].isna()) & (round(df['odometer'], -4) == round(distance, -4)), 'condition'] = avg_condition  
    
print('Количество пропусков в столбце condition:', df['condition'].isna().sum())

Количество пропусков в столбце condition: 11784
Количество пропусков в столбце condition: 0


In [190]:
# заполним пропуски в столбце color самым популярным значением по модели:
print('Количество пропусков в столбце color:', df['color'].isna().sum())

for model in df[df['color'].isna()]['model'].unique():
    try:
        mode_color = df.loc[df['model'] == model, 'color'].mode()[0]
        df.loc[(df['color'].isna()) & (df['model'] == model), 'color'] = mode_color
    except:
        pass
    
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[['color']])
df[['color']] = imputer.transform(df[['color']])

print('Количество пропусков в столбце color:', df['color'].isna().sum())

Количество пропусков в столбце color: 25212
Количество пропусков в столбце color: 0


In [191]:
# заполним пропуски в столбце interior самым популярным значением по модели:
print('Количество пропусков в столбце interior:', df['interior'].isna().sum())

for model in df[df['interior'].isna()]['model'].unique():
    try:
        mode_interior = df.loc[df['model'] == model, 'interior'].mode()[0]
        df.loc[(df['interior'].isna()) & (df['model'] == model), 'interior'] = mode_interior
    except:
        pass
    
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[['interior']])
df[['interior']] = imputer.transform(df[['interior']])

print('Количество пропусков в столбце interior:', df['interior'].isna().sum())

Количество пропусков в столбце interior: 17687
Количество пропусков в столбце interior: 0


In [192]:
# заполним пропуски в столбце body наиболее популярным типом кузова для модели:
print('Количество пропусков в столбце body:', df['body'].isna().sum())

body_dict = df.groupby(['model', 'body'], as_index=False).agg({'vin':'count'}).sort_values(by='vin', ascending=False)
body_dict = body_dict.drop_duplicates(subset='model', keep='first')
df.loc[df['body'].isna(), 'body'] = df[df['body'].isna()]['model'].map(dict(zip(body_dict['model'], body_dict['body'])))

print('Количество пропусков в столбце body:', df['body'].isna().sum())

Количество пропусков в столбце body: 12987
Количество пропусков в столбце body: 2852


In [193]:
# расчитаем расстояние Левенштейна и объединим схожих по написанию производителей:
columns = ['make']
data = []
categorical_dict = []

for column in columns:
    data = pd.DataFrame(df[column].unique()).dropna()
    data.columns = [column]
    for i in data[column].unique():
        data[i] = data[column].apply(lambda x:fuzz.ratio(x, i) >= 80)
        min_distance = np.min(data[data[i]==True][column])
        data.loc[data[column]==i, 'group'] = min_distance
        data = data[[f"{column}", 'group']]
    categorical_dict.append(dict(zip(data[column], data['group'])))
    
df['make'] = df['make'].map(categorical_dict[0])

# унифицируем значения в столбце make:
df.loc[df['make'].str.contains('fordtruck', na=False), 'make'] = 'ford' 
df.loc[df['make'].str.contains('chevtruck', na=False), 'make'] = 'chevrolet'
df.loc[df['make'].str.contains('gmctruck', na=False), 'make'] = 'gmc'
df.loc[df['make'].str.contains('vw', na=False), 'make'] = 'volkswagen'
df.loc[df['make'].str.contains('mercedes-benz', na=False), 'make'] = 'mercedes'

In [194]:
# расчитаем расстояние Левенштейна и объединим схожие по написанию модели:
columns = ['model']
data = []
categorical_dict = []

for column in columns:
    data = pd.DataFrame(df[column].unique()).dropna()
    data.columns = [column]
    for i in data[column].unique():
        data[i] = data[column].apply(lambda x:fuzz.ratio(x, i) >= 90)
        min_distance = np.min(data[data[i]==True][column])
        data.loc[data[column]==i, 'group'] = min_distance
        data = data[[f"{column}", 'group']]
    categorical_dict.append(dict(zip(data[column], data['group'])))
    
df['model'] = df['model'].map(categorical_dict[0])

In [195]:
# изучим оставшиеся пропуски:
df.isna().sum()

data_type            0
year                 0
make                 0
model              176
trim               348
body              2852
transmission         0
vin                  0
state                0
condition            0
odometer             0
color                0
interior             0
seller               0
sellingprice    110060
saledate             0
sale_year            0
sale_month           0
dtype: int64

In [196]:
# заполним пропуски в столбцах trim и body наиболее часто встречающимися значениями:
imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[['trim']])
df[['trim']] = imputer.transform(df[['trim']])

imputer = SimpleImputer(strategy='most_frequent', missing_values=np.nan)
imputer = imputer.fit(df[['body']])
df[['body']] = imputer.transform(df[['body']])

# заполним пропуски  в столбце model:
df.loc[df['model'].isna(), 'model'] = 'undefined'

## Обучение модели

In [166]:
# разделим датафрейм на обучающую и тестовую выборки:
train = df[df['data_type']=='train'].drop('data_type', axis=1).reset_index(drop=True)
test = df[df['data_type']=='test'].drop('data_type', axis=1).reset_index(drop=True)

# разделим выборки на матрицу признаков X и вектор целевой переменной y:
X_train = train.drop(['sellingprice', 'vin', 'seller', 'saledate'], axis=1)
y_train = train['sellingprice']
X_test = test.drop(['sellingprice', 'vin', 'seller', 'saledate'], axis=1)

In [167]:
# используем порядковое кодирование категориальных переменных:
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)

# обучаем энкодер:
encoder.fit(X_train[cat_features])

# применяем кодировку:
X_train[encoder.get_feature_names_out()] = encoder.transform(X_train[cat_features])
X_test[encoder.get_feature_names_out()] = encoder.transform(X_test[cat_features])

In [169]:
# зададим гиперпараметры для модели:
n_estimators = np.arange(10, 110, 10)
max_depth = np.arange(1, 11, 1)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]

random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               }

In [42]:
# обучим модель:
gbr = GradientBoostingRegressor(random_state=42, loss='absolute_error')

model = RandomizedSearchCV(estimator=gbr, 
                           param_distributions=random_grid,
                           n_iter = 10, 
                           cv = ShuffleSplit(test_size=0.20, n_splits=1, random_state=42), 
                           verbose=2,
                           return_train_score=True,
                           random_state=42, 
                           n_jobs = -1)

model.fit(X_train, y_train)

Fitting 1 folds for each of 10 candidates, totalling 10 fits
[CV] END max_depth=1, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   0.3s
[CV] END max_depth=1, min_samples_leaf=4, min_samples_split=5, n_estimators=10; total time=   3.8s
[CV] END max_depth=1, min_samples_leaf=4, min_samples_split=5, n_estimators=30; total time=  10.9s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=10; total time=   0.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=10, n_estimators=20; total time=  17.2s


RandomizedSearchCV(cv=ShuffleSplit(n_splits=1, random_state=42, test_size=0.2, train_size=None),
                   estimator=GradientBoostingRegressor(loss='absolute_error',
                                                       random_state=42),
                   n_jobs=-1,
                   param_distributions={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100])},
                   random_state=42, return_train_score=True, verbose=2)

In [52]:
# оценим качество модели по метрике MAPE:
y_pred_train = model.predict(X_train)
print('MAPE:', mean_absolute_percentage_error(y_train, y_pred_train))

MAPE: 0.19537506346406475
[CV] END max_depth=9, min_samples_leaf=1, min_samples_split=5, n_estimators=20; total time=   0.3s
[CV] END max_depth=2, min_samples_leaf=1, min_samples_split=10, n_estimators=10; total time=   6.2s
[CV] END max_depth=1, min_samples_leaf=4, min_samples_split=10, n_estimators=70; total time=  22.0s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=  31.7s
[CV] END max_depth=1, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=9, min_samples_leaf=1, min_samples_split=5, n_estimators=20; total time=  43.3s
[CV] END max_depth=3, min_samples_leaf=2, min_samples_split=10, n_estimators=20; total time=   0.3s
[CV] END max_depth=1, min_samples_leaf=4, min_samples_split=5, n_estimators=30; total time=   0.1s
[CV] END max_depth=4, min_samples_leaf=1, min_samples_split=10, n_estimators

In [ ]:
# используем модель для прогноза стоимости на тестовой выборке:
y_pred = model.predict(X_test)
predictions = pd.DataFrame(y_pred)

In [ ]:
# выгрузим прогноз:
test_df = pd.read_csv('/Users/a.babaev/Desktop/kaggle/test.csv')
test_df = test_df[['vin']]
final_df = test_df.join(predictions)
final_df.columns = ['vin', 'sellingprice']
final_df.to_csv('submission.csv', index=False)